In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [2]:
with open("data-files/nietzsche.txt", "rt") as f:
    nietzsche_text = f.read()

In [3]:
print( len(nietzsche_text) )
nietzsche_text[:30]

600893


'PREFACE\n\n\nSUPPOSING that Truth'

In [4]:
# 대문자 -> 소문자
nietzsche_lower_text = nietzsche_text.lower()
nietzsche_lower_text[:30]

'preface\n\n\nsupposing that truth'

In [5]:
# 전체 텍스트에 포함된 문자 확인
print( np.unique(list(nietzsche_lower_text)) )
print( np.unique(list(nietzsche_lower_text)).shape )

['\n' ' ' '!' '"' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7'
 '8' '9' ':' ';' '=' '?' '[' ']' '_' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i'
 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' 'ä'
 'æ' 'é' 'ë']
(57,)


In [6]:
# 문자 사전 만들기

set(nietzsche_lower_text) # set : 중복되지 않는 리스트
sorted_chars = sorted( set(nietzsche_lower_text) )
print( sorted_chars )

char_to_idx = { ch:i for i, ch in enumerate(sorted_chars) } # 문자 : 숫자
print( char_to_idx )

idx_to_char = { i:ch for ch, i in char_to_idx.items() }     # 숫자 : 문자
print( idx_to_char )

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ä', 'æ', 'é', 'ë']
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '4', 15: '5', 16: '6', 17: '7', 18: '8', 19: '9', 20: ':', 21: ';', 22: '=', 23: '?', 

In [7]:
# 학습 내용

# n개의 연속된 문자 -> n+1번째 문자 예측
# (입력데이터)        (출력데이터, target)

In [8]:
# 학습 데이터 준비

sequence_length = 50    # 연속된 문자 갯수
step = 3                # stride (3문자씩 이동하면서 데이터 추출)

sequences = []      # (batch크기, 입력문자갯수, 단어사전크기)
next_chars = []     # (batch크기, 단어사전크기)

for idx in range(0, len(nietzsche_lower_text) - sequence_length, step):
    sequences.append(nietzsche_lower_text[idx:idx+sequence_length])
    next_chars.append(nietzsche_lower_text[idx+sequence_length])

# print( len(sequences), len(next_chars) )
# print( sequences[0], next_chars[0])

X = np.zeros(shape=(len(sequences), sequence_length, len(sorted_chars)))
y = np.zeros(shape=(len(sequences), len(sorted_chars)))

for si, sequence in enumerate(sequences):   # si : 입력 문장 순서 번호
    # print(si, sequence)
    for ci, ch in enumerate(sequence):      # ci: 한 개의 입력문장 안의 문자 순서 번호
        X[si, ci, char_to_idx[ch]] = 1
        y[si, char_to_idx[next_chars[si]]] = 1
        # print(ci, ch, end=", ")

In [43]:
# 모델 구조 설계

input = tf_keras.layers.Input(shape=(sequence_length, len(sorted_chars))) # (50, 57)
x = tf_keras.layers.LSTM(units=128)(input)
output = tf_keras.layers.Dense(units=len(sorted_chars), activation="softmax")(x)

model = tf_keras.models.Model(input, output)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50, 57)]          0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               95232     
                                                                 
 dense_2 (Dense)             (None, 57)                7353      
                                                                 
Total params: 102585 (400.72 KB)
Trainable params: 102585 (400.72 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
# 모델 학습 설계
model.compile(loss="categorical_crossentropy",
              optimizer=tf_keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [45]:
history = model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50
1565/1565 [==============================] - 92s 57ms/step - loss: 2.0344 - accuracy: 0.4053
Epoch 2/50
1565/1565 [==============================] - 89s 57ms/step - loss: 1.6622 - accuracy: 0.5049
Epoch 3/50
1565/1565 [==============================] - 81s 52ms/step - loss: 1.5565 - accuracy: 0.5316
Epoch 4/50
1565/1565 [==============================] - 81s 52ms/step - loss: 1.4994 - accuracy: 0.5457
Epoch 5/50
1565/1565 [==============================] - 83s 53ms/step - loss: 1.4640 - accuracy: 0.5548
Epoch 6/50
1565/1565 [==============================] - 81s 52ms/step - loss: 1.4383 - accuracy: 0.5617
Epoch 7/50
1565/1565 [==============================] - 88s 56ms/step - loss: 1.4175 - accuracy: 0.5654
Epoch 8/50
1565/1565 [==============================] - 80s 51ms/step - loss: 1.4029 - accuracy: 0.5707
Epoch 9/50
1565/1565 [==============================] - 78s 50ms/step - loss: 1.3906 - accuracy: 0.5737
Epoch 10/50
1565/1565 [==============================] - 78s 50m

In [47]:
model.save('models/generation-model.keras')

In [10]:
loaded_model = tf_keras.models.load_model('models/generation-model.keras')

In [12]:
def select_character(preds, temperature=1.0): # temperature 값이 작을 수록 낮은 확률의 값이 선택 가능성이 낮짐
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)    
    probas = np.random.multinomial(1, preds, 1) # 주어진 확률에 따라 다음 값 랜덤 선택
    return np.argmax(probas)

In [15]:
start_idx = np.random.randint(0, len(nietzsche_lower_text) - sequence_length)
seed_text = nietzsche_lower_text[start_idx:start_idx + sequence_length]

for temperature in [0.5, 1.0, 1.5]:

    full_text = seed_text

    print("temperature : {0}".format(temperature))
    print(seed_text, end="")

    for idx in range(100):
        sample = np.zeros(shape=(1, sequence_length, len(sorted_chars))) # 1, 50, 57
        for ci, c in enumerate(seed_text):
            sample[0, ci, char_to_idx[c]] = 1

        # predicted_values = model.predict(sample, verbose=0)
        predicted_values = loaded_model.predict(sample, verbose=0)
        
        # selected_char_idx = predicted_values[0].argmax()
        selected_char_idx = select_character(predicted_values[0], temperature)
        
        full_text += sorted_chars[selected_char_idx]
        seed_text = full_text[idx+1:]

        print(sorted_chars[selected_char_idx], end="")

    print("\n\n")
    print("=" * 50)

temperature : 0.5
s no demonstrable supremacy and a struggle leads by the

C:\Users\human\AppData\Local\Temp\ipykernel_16612\47405372.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 european and more protect of secret in the struggnes of
the most believed to be always partica


temperature : 1.0
truggnes of
the most believed to be always partical or capaburd beger attains dinglers of the unchumen foreve_, and therefore-anith, dresent which a g


temperature : 1.5
en foreve_, and therefore-anith, dresent which a good-self homeci." wout
most a       it been pein too, justive is crune? and late!

128

=the
trocabl


